##### Installing Required Libraries : 

EUMDAC library will be installed. EUMDAC is the EUMETSAT Data Access Client. It provides simple access to the EUMETSAT data of all satellite missions. As a Python library, it comes with many methods and helpers to use EUMETSATs APIs and services, like Data Store. 

In [ ]:
# ! pip install eumdac
# ! pip install requests
# ! pip install patool

##### Importing Required Libraries

In [ ]:
import os
import time
import datetime
import shutil
import requests
import eumdac
import patoolib
import json

for importing utils in the parent directory we add that to the python path as below

In [ ]:
import sys

# Get the parent directory of the current script
parent_dir = os.path.dirname(os.path.abspath('Seviri_Download.ipynb'))
parent_dir = os.path.abspath(os.path.join(parent_dir, ".."))  # Go up one level

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from utils import extract_zip_files, read_credentials_from_config

##### Downloading latest files

Insert your personal key and secret into the single quotes. https://eoportal.eumetsat.int/

In [ ]:
consumer_key, consumer_secret = read_credentials_from_config(filename='../config.json')



credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

1-Discovering **Collections**

collection identifier - collection title
  - EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree 
  - EO:EUM:DAT:MSG:HRSEVIRI-IODC - High Rate SEVIRI Level 1.5 Image Data - MSG - Indian Ocean


In [ ]:
seviri_pdcid = "EO:EUM:DAT:MSG:HRSEVIRI" # prime data collection id
seviri_iodcid = "EO:EUM:DAT:MSG:HRSEVIRI-IODC" # indian ocean data collection id

Data Collections based on collection identifiers

In [ ]:
datastore = eumdac.DataStore(token)
seviri_pdc = datastore.get_collection(seviri_pdcid) # prime data collection
seviri_iodc = datastore.get_collection(seviri_iodcid) # indian ocean data collection

2-Searching and Filtering to find required **Products**/**Datasets**

Filtering by latest product

In [ ]:
seviri_pp_latest = seviri_pdc.search().first() # prime product_latest
seviri_iop_latest = seviri_iodc.search().first() # indian ocean product_latest
products = [seviri_pp_latest, seviri_iop_latest]

3-Downloading **Products**

 Products can be downloaded by providing either their product ID, or a combination of their collection ID and the time of acquisition. We can download entire products, or specific file components (e.g. metadata only).

In [ ]:
# Create the "tmp" directory if it doesn't exist
base_dir = './tmp'
os.makedirs(base_dir, exist_ok=True)

for product in products:
    try:
        with product.open() as fsrc, open(os.path.join(base_dir, os.path.basename(fsrc.name)), mode='wb') as fdst:
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {product} finished.')
    except eumdac.product.ProductError as error:
        print(f"Error related to the product '{product}' while trying to download it: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    
print('All downloads are finished.')

Then we will extract archived file.

In [ ]:
extract_zip_files(base_dir)